# [Build an Agent](https://python.langchain.com/docs/tutorials/agents/)

エージェントは、LLM を推論エンジンとして使用して、実行するアクションと、アクションを実行するために必要な入力を決定するシステムです。  
アクションを実行した後、結果を LLM にフィードバックして、さらにアクションが必要かどうか、または終了しても問題ないかを判断できます。

## LangSmith

https://smith.langchain.com/

## Tavily

https://tavily.com/


## リファレンス

- [TavilySearchResults](https://python.langchain.com/api_reference/community/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html)


# Tavilyの定義

In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Tokyo")
print(search_results)

tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1730643303, 'localtime': '2024-11-03 23:15'}, 'current': {'last_updated_epoch': 1730643300, 'last_updated': '2024-11-03 23:15', 'temp_c': 18.1, 'temp_f': 64.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 115, 'wind_dir': 'ESE', 'pressure_mb': 1022.0, 'pressure_in': 30.18, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 68, 'cloud': 0, 'feelslike_c': 18.1, 'feelslike_f': 64.6, 'windchill_c': 17.6, 'windchill_f': 63.7, 'heatindex_c': 17.6, 'heatindex_f': 63.7, 'dewpoint_c': 8.6, 'dewpoint_f': 47.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 9.4, 'gust_kph': 15.2}}"}, {'url': 'https://weatherspark.com/h/m/143809/2024/3/Historical-Weather-in-March-2024-in-Toky

# 言語モデル

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4")

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

モデルからツールを呼び出す

In [21]:
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Tokyo?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Tokyo'}, 'id': 'call_nzPbitBx0mGCsun0mb2QtKPU', 'type': 'tool_call'}]


# エージェントを作成する

In [22]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(model, tools)

# エージェントを実行する

In [24]:
response = agent_executor.invoke({"messages": [HumanMessage(content="What's the weather in Tokyo?")]})
response["messages"]

[HumanMessage(content="What's the weather in Tokyo?", additional_kwargs={}, response_metadata={}, id='12afc33f-1689-4963-94d5-2f6b0983a879'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NOGn1xVrW5WCz7cTqRvrAtTz', 'function': {'arguments': '{\n  "query": "current weather in Tokyo"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-61946716-34a8-49fe-a47e-2dcc0534a6c6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Tokyo'}, 'id': 'call_NOGn1xVrW5WCz7cTqRvrAtTz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_to

# ストリーミング

In [25]:
output_stream = agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Tokyo?")]}
)
for chunk in output_stream:
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RekNETi8JTXz2y2FlZ8uFD6r', 'function': {'arguments': '{\n  "query": "current weather in Tokyo"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4fab70ee-94d6-4da6-8fa4-7888618a1f77-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Tokyo'}, 'id': 'call_RekNETi8JTXz2y2FlZ8uFD6r', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 22, 'total_tokens': 110, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}

# トークンストリーミング

In [26]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1730644006, \'localtime\': \'2024-11-03 06:26\'}, \'current\': {\'last_updated_epoch\': 1730643300, \'last_updated\': \'2024-11-03 06:15\', \'temp_c\': 12.8, \'temp_f\': 55.0, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 9.8, \'wind_kph\': 15.8, \'wind_degree\': 323, \'wind_dir\': \'NW\', \'pressure_mb\': 1017.0, \'pressure_in\': 30.04, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 86, \'cloud\': 25, \'feelslike_c\': 11.3, \'feelslike_f\': 52.3, \'

# メモリに追加

In [27]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [28]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a69cf850-317e-4e0c-b396-77a0c2c75e0f-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
----


In [29]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 108, 'total_tokens': 114, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-15d65bec-a6e6-4db0-a015-36fef8401232-0', usage_metadata={'input_tokens': 108, 'output_tokens': 6, 'total_tokens': 114, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
----


In [31]:
# 新しい会話スレッドを開始
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I'm designed to respect user privacy and confidentiality.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 142, 'total_tokens': 186, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4b6067b6-a0ec-4f6a-9627-7c8926539719-0', usage_metadata={'input_tokens': 142, 'output_tokens': 44, 'total_tokens': 186, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
----
